In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import numpy as np
import tifffile as tiff
import os
import cv2
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
import os
from PIL import Image
from keras.layers import Dense,Flatten,Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import EfficientNetB0
from keras.models import Sequential
from keras.applications.vgg16 import preprocess_input
from keras import layers, models
from keras.applications.resnet50 import ResNet50

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
batch_size = 32
img_width=224
img_height=224
img_size=128
epochs=50
NUM_CLASSES=2

In [ ]:
data_generator = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    rotation_range = 180,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    validation_split = 0.2)

train_generator = data_generator.flow_from_directory(
    '/content/drive/MyDrive/deep learning/train',
    target_size =(img_width, img_height),
    batch_size = batch_size,
    shuffle = True,
    #class_mode = 'binary',
    seed = 42,
    subset='training')

validation_generator = data_generator.flow_from_directory(
    '/content/drive/MyDrive/deep learning/test',
    target_size =(img_width, img_height),
    batch_size = batch_size,
    shuffle = True,
   # class_mode = 'binary',
    seed = 42,
    subset='validation')

Found 84 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [ ]:
import tqdm
train_generator.reset()
X_train, y_train = next(train_generator)
for i in tqdm.tqdm(range(int(train_generator.n/batch_size)-1)):
  img, label = next(train_generator)
  X_train = np.append(X_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)

In [ ]:
X_test, y_test = next(validation_generator)
for i in tqdm.tqdm(range(int(train_generator.n/batch_size)-1)):
  img, label = next(train_generator)
  X_train = np.append(X_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator with desired augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Function to apply augmentation twice to each dataset
def augment_dataset(dataset):
    augmented_data = []
    for img in dataset:
        img = np.expand_dims(img, axis=0)  # Expanding dimensions for batch size
        for _ in range(1):  # Apply augmentation twice
            for batch in datagen.flow(img, batch_size=1):
                augmented_data.append(np.squeeze(batch))  # Remove the batch dimension
                break  # Stop the loop after one augmented image
    return np.array(augmented_data)

In [ ]:
# Apply augmentation twice to X_train, X_test, and X_val
X_train_augmented = augment_dataset(X_train)
X_test_augmented = augment_dataset(X_test)
#X_val_augmented = augment_dataset(X_val)
y_train_augmented = np.repeat(y_train, 1, axis=0)  # Adjust based on augmentation factor
y_test_augmented = np.repeat(y_test, 1, axis=0)  # Adjust based on augmentation factor

In [ ]:
from sklearn.utils import shuffle

# Combine original data with augmented data
X_train_combined = np.concatenate((X_train, X_train_augmented))
y_train_combined = np.concatenate((y_train, y_train_augmented))

X_test_combined = np.concatenate((X_test, X_test_augmented))
y_test_combined = np.concatenate((y_test, y_test_augmented))


# Shuffle the combined data
X_train_combined, y_train_combined = shuffle(X_train_combined, y_train_combined, random_state=42)
X_test_combined, y_test_combined = shuffle(X_test_combined, y_test_combined, random_state=42)

In [ ]:
from keras.applications import vgg19


img_rows, img_cols = 224, 224

resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))


for layer in resnet50_base.layers:
    layer.trainable = False
# Let's print our layers
for (i,layer) in enumerate(resnet50_base.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

from keras.models import Model
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dropout(0.1)(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dropout(0.1)(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dropout(0.1)(top_model)
    top_model = Dense(num_classes,activation='sigmoid')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

from keras.models import Model


num_classes = 2

FC_Head = lw(resnet50_base, num_classes)


model = Model(inputs = resnet50_base.input, outputs = FC_Head)

In [ ]:
from keras.models import Model
import tensorflow as tf
from keras import backend as K
metrics = [
        'accuracy',
        tf.keras.metrics.AUC(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.SpecificityAtSensitivity(0.5),
        tf.keras.metrics.SensitivityAtSpecificity(0.5),
        tf.keras.metrics.FalseNegatives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.TruePositives(),]

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',patience=3,
         restore_best_weights=True,
    )
]
history = model.fit(X_train_combined, y_train_combined, epochs=50,batch_size=32,validation_split=0.2,callbacks=callbacks)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
acc = history.history['accuracy']
val_acc= history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,acc,'r',label="Training accuracy")
plt.plot(epochs,val_acc,'b',label="Validation accuracy")
plt.title('Training and Validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.title('Training and Validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()